W C# 10.0 pola w strukturach mogą mieć initializery.\
W C# 11.0 pojawiają się pola ``required`` oraz automatyczna inicjalizacja pól wartością ``default``.\
Dawniej wszystkie pola struktury musiały być uzupełnione przed wyjściem z konstruktora.\
Teraz pola nieustawione _explicite_ w konstruktorze dostają wartość ``default``

In [12]:
struct Struct
{        
    public Struct() { /*...*/ }
    public int[] Ids = { 5, 10, 15 };     
    public /*required*/ string[] Texts;
}

Pole typu array zajmuje w obiekcie nadrzędnym tyle miejsca co referencja (adres). \
Poruszanie się po drzewie takich danych wymaga częstych skoków w pamięci. \
W wymagających zastosowaniach możemy chcieć wstawić wartości bezpośrednio do struktury:
- -> mniej skoków po pamięci 
- -> mniej _CPU cache fills_

Upakowanie struktury da się osiągnąć za pomocą ``fixed``, ale to wymaga ``unsafe``:

In [13]:
unsafe struct Struct1
{
    public fixed int InternalBuffer[16];    
    public Struct1() {
        for (int i=0; i < 16; ++i)
            InternalBuffer[i] = i ^ 0xFF;
    }
}


W C# 12.0 jest też atrybut na 1-elementowej strukturze. Dziwaczne, ale nie wymaga ``unsafe``.\
Warto zauważyć, że pole w tej strukturze jest tylko placeholder na odpowiedni kawałek pamięci,\
ale nie używamy jego nazwy. Do struktury odwołujemy się jakby to był array ``this[i]`` oraz ``s2[i]``.

In [14]:

[System.Runtime.CompilerServices.InlineArray(16)]
struct Struct2 {
    public int SoleAndOnlyMemberWithAbsolutellyIrrelevantName;
    public Struct2() {
        for (int i=0; i < 16; ++i)
            this[i] = i ^ 0xFF;
    }
}

var s2 = new Struct2();
for (int i=0; i < 16; ++i)
    Console.WriteLine(s2[i]);



255
254
253
252
251
250
249
248
247
246
245
244
243
242
241
240


In [15]:

static int SizeOf(Type t) => System.Runtime.InteropServices.Marshal.SizeOf(t);

Console.WriteLine($"sizeof(Struct) = {SizeOf(typeof(Struct))}");   // 2 x sizeof(pointer) = 16
Console.WriteLine($"sizeof(Struct1) = {SizeOf(typeof(Struct1))}"); // 16 x sizeof(int) = 64
Console.WriteLine($"sizeof(Struct1) = {SizeOf(typeof(Struct2))}"); // 16 x sizeof(Struct2) = 16 x sizeof(int) = 64

sizeof(Struct) = 16
sizeof(Struct1) = 64
sizeof(Struct1) = 64
